# Modeling and Simulation in Python

Chapter 2:

Copyright 2017 Allen Downey

License: [Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0)


In [44]:
# If you want the figures to appear in the notebook, use
# %matplotlib notebook

# If you want the figures to appear in separate windows, use
# %matplotlib qt

# To switch from one to another, you have to select Kernel->Restart

%matplotlib notebook

from modsim import *

In [45]:
def run_steps(state, sum_steps=1, p1=0.5, p2=0.5):
    """Simulate the given number of time steps.
    
    state: bikeshare State object
    sum_steps: number of time steps
    p1: probability of an Olin->Wellesley customer arrival
    p2: probability of a Wellesley->Olin customer arrival
    """
    for i in range(sum_steps):
        step(state, p1, p2)
        plot_state(state)
        
def step(state, p1=0.5, p2=0.5):
    """Simulate one minute of time.
    
    state: bikeshare State object
    p1: probability of an Olin->Wellesley customer arrival
    p2: probability of a Wellesley->Olin customer arrival
    """
    if flip(p1):
        bike_to_wellesley(state)
    
    if flip(p2):
        bike_to_olin(state)
        
def bike_to_wellesley(state):
    """Move one bike from Olin to Wellesley.
    
    state: bikeshare State object
    """
    move_bike(state, 1)
    
def bike_to_olin(state):
    """Move one bike from Olin to Wellesley.
    
    state: bikeshare State object
    """
    move_bike(state, -1)
    
def move_bike(state, n):
    """Move a bike.
    
    state: bikeshare State object
    n: +1 to move from Olin to Wellesley or
       -1 to move from Wellesley to Olin
    """
    state.olin -= n
    state.wellesley += n
    
def plot_state(state):
    """Plot the current state of the bikeshare system.
    
    state: bikeshare State object
    """
    plot(state.olin, 'rs-', label='Olin')
    plot(state.wellesley, 'bo-', label='Wellesley')
    
def annotate():
    """Add a legend and label the axes.
    """
    legend(loc='best')
    label_axes(title='Olin-Wellesley Bikeshare',
               xlabel='Time step (min)', 
               ylabel='Number of bikes')

In [57]:
bikeshare1 = State(olin=10, wellesley=2)
bikeshare1

olin -> 10
wellesley -> 2

In [58]:
bikeshare2 = State(olin=2, wellesley=10)
bikeshare2

olin -> 2
wellesley -> 10

In [59]:
bike_to_olin(bikeshare1)

In [60]:
bike_to_wellesley(bikeshare2)

In [61]:
bikeshare1

olin -> 11
wellesley -> 1

In [62]:
bikeshare2

olin -> 1
wellesley -> 11

## Negative bikes

In [3]:
bikeshare = State(olin=10, wellesley=2)
newfig()
plot_state(bikeshare)
annotate()
run_steps(bikeshare, 60, 0.4, 0.2)

In [4]:
def move_bike(state, n):
    # make sure the number of bikes won't go negative
    olin = state.olin - n
    if olin < 0:
        return
    
    wellesley = state.wellesley + n
    if wellesley < 0:
        return
    
    # update the state
    state.olin = olin
    state.wellesley = wellesley

In [5]:
bikeshare = State(olin=10, wellesley=2)
newfig()
plot_state(bikeshare)
annotate()
run_steps(bikeshare, 60, 0.4, 0.2)

Try to access a local variable from outside the function.

In [63]:
olin

NameError: name 'olin' is not defined

## Metrics

In [6]:
bikeshare = State(olin=10, wellesley=2, 
                  olin_empty=0, wellesley_empty=0)

In [7]:
def move_bike(state, n):
    olin = state.olin - n
    if olin < 0:
        state.olin_empty += 1
        return
    
    wellesley = state.wellesley + n
    if wellesley < 0:
        state.wellesley_empty += 1
        return
    
    state.olin = olin
    state.wellesley = wellesley

In [8]:
newfig()
plot_state(bikeshare)
annotate()
run_steps(bikeshare, 60, 0.4, 0.2)

In [9]:
bikeshare.olin_empty

2

In [10]:
bikeshare.wellesley_empty

0

**Exercise:** Let's add a "clock" to keep track of how many time steps have elapsed.  Add a new state variable named `clock` to `bikeshare`, initialized to 0, and modify `step` so it increments (adds one to) `clock` each time it is invoked.

Test your code by adding a print statement that prints the value of `clock` at the beginning of each time step.

**Exercise:** Now suppose we'd like to know how long it takes to run out of bikes at either location.  Modify `move_bike` so the first time a student arrives and doesn't find a bike, it records the value of `clock` in a state variable.

Hint: create a state variable named `t_first_empty` and initialize it to `None`, which is a special value (like `True` and `False`) that can be used to indicate a "special case".

Test your code by running a simulation for 60 minutes and checking the 

## Returning values

In [11]:
def add_five(x):
    return x + 5

In [12]:
add_five(3)

8

In [13]:
def run_steps(state, sum_steps=1, p1=0.5, p2=0.5, plot=True):
    """Simulate the given number of time steps.
    
    state: bikeshare State object
    sum_steps: number of time steps
    p1: probability of an Olin->Wellesley customer arrival
    p2: probability of a Wellesley->Olin customer arrival
    plot: boolean, whether to plot
    """
    for i in range(sum_steps):
        step(state, p1, p2)
        if plot:
            plot_state(state)

In [14]:
bikeshare = State(olin=10, wellesley=2, 
                  olin_empty=0, wellesley_empty=0)
run_steps(bikeshare, 60, 0.4, 0.2, plot=False)

In [15]:
bikeshare.olin_empty

2

In [16]:
def run_simulation():
    bikeshare = State(olin=10, wellesley=2, 
                  olin_empty=0, wellesley_empty=0)
    run_steps(bikeshare, 60, 0.4, 0.2, plot=False)
    return bikeshare

In [17]:
run_simulation()

olin_empty -> 4
wellesley_empty -> 0
olin -> 1
wellesley -> 11

In [18]:
def run_simulation(p1=0.4):
    bikeshare = State(olin=10, wellesley=2, 
                  olin_empty=0, wellesley_empty=0)
    run_steps(bikeshare, 60, p1, 0.2, plot=False)
    return bikeshare

In [19]:
state = run_simulation(0.2)
state.olin_empty

0

In [20]:
state = run_simulation(0.6)
state.olin_empty

0

**Exercise:**  Write a version of `run_simulation` that takes both `p1` and `p2` as parameters.

## More for loops

In [21]:
for i in range(4):
    print(i)

0
1
2
3


In [22]:
from numpy import linspace

In [23]:
linspace(0, 1, 11)

array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ])

In [24]:
p1_array = linspace(0, 1, 11)

for p1 in p1_array:
    print(p1)

0.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0


In [25]:
for p1 in p1_array:
    state = run_simulation(p1)
    print(p1, state.olin_empty)

0.0 0
0.1 0
0.2 0
0.3 0
0.4 6
0.5 0
0.6 20
0.7 20
0.8 25
0.9 31
1.0 39


New version of `plot`

In [26]:
newfig()
for p1 in p1_array:
    state = run_simulation(p1)
    plot(p1, state.olin_empty, 'rs', label='olin')

In [27]:
def annotate():
    legend(loc='upper left')
    label_axes(title='Olin-Wellesley Bikeshare',
               xlabel='Arrival rate at Olin (customers/min)', 
               ylabel='Number of unhappy customers')

In [28]:
annotate()

In [29]:
def parameter_sweep(p1_array):
    newfig()
    for p1 in p1_array:
        state = run_simulation(p1)
        plot(p1, state.olin_empty, 'rs', label='olin')
        plot(p1, state.wellesley_empty, 'bo', label='wellesley')

In [30]:
p1_array = linspace(0, 1, 101)
parameter_sweep(p1_array)
annotate()

**Exercise:** Modify `parameter_sweep` so it also plots `wellesley_empty` for each value of `p1`.

## Bigbelly

In [ ]:
https://www.youtube.com/watch?v=frix_zTkPEs

In [31]:
from pysolar.solar import *

In [32]:
from datetime import datetime, timedelta

In [33]:
dt = datetime.now()

In [34]:
get_altitude(42.2931671, -71.263665, dt)

/home/downey/anaconda2/envs/py3k/lib/python3.5/site-packages/pysolar/time.py:105: UserWarning: I don't know about leap seconds after 2015
  (leap_seconds_base_year + len(leap_seconds_adjustments) - 1)


70.25111551534773

In [35]:
latitude_deg = 42.3
longitude_deg = -71.3
d = datetime(2007, 2, 18, 15, 13, 1, 130320)
altitude_deg = get_altitude(latitude_deg, longitude_deg, d)
azimuth_deg = get_azimuth(latitude_deg, longitude_deg, d)
radiation.get_radiation_direct(d, altitude_deg)

# result is in Watts per square meter

792.1238352496164

In [36]:
location = State(lat_deg=42.3, lon_deg=-71.3)
print_state(location)

lon_deg -> -71.3
lat_deg -> 42.3


In [37]:
dt = datetime(year=2017, month=9, day=15, hour=12, minute=30)
str(dt)

'2017-09-15 12:30:00'

In [38]:
def compute_irradiance(location, dt):
    degree = UNITS.degree
    watt = UNITS.watt
    meter = UNITS.meter
    
    sun = State()
    sun.altitude_deg = get_altitude(location.lat_deg, location.lon_deg, dt)
    sun.azimuth_deg = get_azimuth(location.lat_deg, location.lon_deg, dt)

    if sun.altitude_deg <= 0:
        irradiance = 0
    else:
        irradiance = radiation.get_radiation_direct(dt, sun.altitude_deg)

    sun.irradiance = irradiance * watt / meter**2
    return sun

In [39]:
dt = datetime(year=2017, month=9, day=15, hour=14, minute=30)
sun = compute_irradiance(location, dt)
sun

/home/downey/anaconda2/envs/py3k/lib/python3.5/site-packages/pysolar/time.py:105: UserWarning: I don't know about leap seconds after 2015
  (leap_seconds_base_year + len(leap_seconds_adjustments) - 1)


altitude_deg -> 43.469828404534525
azimuth_deg -> -39.36678521291222
irradiance -> 865.7572442289041 watt / meter ** 2

In [40]:
newfig()

dt = datetime(year=2017, month=9, day=15)
delta_t = timedelta(minutes=15)
delta_t

for i in range(24 * 4):
    dt += delta_t
    sun = compute_irradiance(location, dt)
    plot(sun.irradiance.magnitude)


/home/downey/anaconda2/envs/py3k/lib/python3.5/site-packages/pysolar/time.py:105: UserWarning: I don't know about leap seconds after 2015
  (leap_seconds_base_year + len(leap_seconds_adjustments) - 1)


In [41]:
cm = UNITS.centimeter
meter = UNITS.meter
watt = UNITS.watt
second = UNITS.second
joule = UNITS.joule

In [42]:
width = 45 * cm

In [73]:
area = width**2
area

<Quantity(2025, 'centimeter ** 2')>

In [74]:
power = sun.irradiance * area
power

<Quantity(0.0, 'centimeter ** 2 * watt / meter ** 2')>

In [75]:
area = area.to(meter**2)

In [76]:
power = sun.irradiance * area
power

<Quantity(0.0, 'watt')>

In [77]:
delta_t = 1 * second
energy = power * delta_t
energy

<Quantity(0.0, 'second * watt')>

In [78]:
energy.to(joule)

<Quantity(0.0, 'joule')>

datetime.timedelta(0, 900)